In [72]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from visualize import PyramidVisualizer, get_dataset, simple_visualizer
from metroem.aligner import Aligner
from collections import defaultdict
import random
import cloudvolume as cv
import numpy as np
import metroem
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
cv_path = 'precomputed://gs://h01-release/data/20210601/4nm_raw'

In [24]:
def get_srctgt(cv_path, x_start, y_start, z, img_mip, x_size=1024, y_size=1024):
    img_cv = cv.CloudVolume(cv_path, mip=img_mip, parallel=True, progress=False, bounded=False, fill_missing=True)
    
    x_size *= 2**img_mip
    y_size *= 2**img_mip

    img1_np = img_cv[(x_start)//2**img_mip:(x_start+x_size)//2**img_mip, (y_start)//2**img_mip:(y_start+y_size)//2**img_mip, z:z+1].squeeze()
    img2_np = img_cv[(x_start)//2**img_mip:(x_start+x_size)//2**img_mip, (y_start)//2**img_mip:(y_start+y_size)//2**img_mip, z+1:z+2].squeeze()

    img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
    img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)
    
    img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
    img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)

    #src_tgt_t = torch.cat([img1_t, img2_t], 1)
    return img1_t, img2_t

In [209]:
img_cv = cv.CloudVolume(cv_path, mip=1, parallel=True, progress=False, bounded=False, fill_missing=True)
size = 4096 * 2

In [210]:
img = np.array(img_cv[223045:223045+size, 130013:130013+size, 1778].squeeze())

In [229]:
img_cv.scales[1]

{'chunk_sizes': [[128, 128, 32]],
 'encoding': 'jpeg',
 'key': '8.0x8.0x33.0',
 'resolution': [8, 8, 33],
 'sharding': {'@type': 'neuroglancer_uint64_sharded_v1',
  'hash': 'identity',
  'minishard_bits': 6,
  'minishard_index_encoding': 'gzip',
  'preshift_bits': 9,
  'shard_bits': 17},
 'size': [515892, 356400, 5293]}

In [230]:
img_t = torch.tensor(img, device='cuda').unsqueeze(0).unsqueeze(0)
for i in range(7 - 1):
    img_t = torch.nn.functional.interpolate(img_t, scale_factor=1.0/(2**1)).float()

In [233]:

img_n = metroem.helpers.normalize(img_t, mask=torch.ones_like(img_t).float())
emb = aligner_m7.aligner.get_embeddings(img_n)
emb.shape

torch.Size([3, 128, 128])

In [232]:
simple_visualizer().visualize([img_n, img_t, img, emb[0]])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [204]:
import modelhouse
import json
aligner_m4 = modelhouse.load_model(
    path="gs://corgie/models/pyramid_m4m6m9/2_mip4in_mip4module",
    params=json.dumps({
         "finetune": True, "finetune_iter": 1, "checkpoint_name": "metric_net",
         "finetune_sm": 10
    })
).cuda()
aligner_m5 = modelhouse.load_model(
    path="gs://corgie/models/pyramid_m4m6m9/1_mip5in_mip6module",
    params=json.dumps({
         "finetune": True, "finetune_iter": 1, "checkpoint_name": "metric_net",
         "finetune_sm": 10
    })
).cuda()
aligner_m7 = modelhouse.load_model(
    path="gs://corgie/models/pyramid_m4m6m9/0_mip7in_mip9module",
    params=json.dumps({
         "finetune": True, "finetune_iter": 1, "checkpoint_name": "metric_net",
         "finetune_sm": 10
    })
).cuda()

/usr/people/popovych/.modelhouse/tmp_files/tmp_z1rjbeq/model/model_spec.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
identity.json
embedder_m0_3x3_fms3x.json
block_5convs_3x3_fms1to3_skip14.json
average_pool.json
embedder_fms4to3.json
block_3convs_3x3_fms4to32to3.json
average_pool.json
block_3convs_3x3_fms3to32to3.json
identity.json
identity.json
identity.json
upsample_residuals.json
identity.json


/usr/people/popovych/miniconda3/envs/corgie_env_x3/lib/python3.8/site-packages/artificery-0.0.3.3-py3.8.egg/artificery/parsers/categorical_regression/parse.py:37: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  self.weights = torch.range(0, self.component_channels - 1, device=device,
